In [1]:
import os
os.environ['CUDA_DEVICE_ORDER']="PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES']='1'

import torch

print(torch.__version__)
print(torch.version.cuda)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name)

2.1.0
11.8
cuda
True
0
<function get_device_name at 0x7facf79fcaf0>


In [ ]:
import os
import pandas as pd

data_path = '../../data/jeehoshin/foodcom_dataset/'
raw_interaction = pd.read_csv(data_path + 'RAW_interactions.csv')
pp_recipe = pd.read_csv(data_path + 'PP_recipes.csv')
raw_recipe = pd.read_csv(data_path + 'RAW_recipes.csv')

records_removed = True
df = raw_interaction[raw_interaction['recipe_id'].isin(pp_recipe['id'].tolist())].reset_index(drop=True)

while records_removed:
    user_counts = df['user_id'].value_counts()
    item_counts = df['recipe_id'].value_counts()

    valid_users = user_counts[user_counts >= 5].index
    valid_items = item_counts[item_counts >= 5].index

    filtered_df = df[df['user_id'].isin(valid_users) & df['recipe_id'].isin(valid_items)].reset_index(drop=True)

    if len(filtered_df) == len(df):
        records_removed = False
    else:
        records_removed = True

    df = filtered_df
    
core_inter = df
sort_filter_ui = core_inter.sort_values('date').reset_index(drop=True)

file_names = os.listdir(data_path + 'images/')

finish_image = [int(os.path.splitext(file_name)[0]) for file_name in file_names]
sort_filter_ui = sort_filter_ui[sort_filter_ui['recipe_id'].isin(finish_image)].reset_index(drop=True)
print(len(finish_image))
sort_filter_ui

29943


,user_id,recipe_id,date,rating,review
0,2999,3567,2000-10-23,5,I have made this pie instead of plain ol' pump...
1,2178,3704,2000-10-30,3,Careful not to cook it too long... you want th...
2,2178,4366,2000-11-04,5,"if you like oysters, this is a great alternati..."
3,5523,7695,2001-02-01,1,I agree.
4,42189,4460,2001-02-12,5,I have had this before. It has really good fla...
...,...,...,...,...,...
428496,2001513060,367414,2018-12-17,1,"maybe I did something wrong , but I thought th..."
428497,2001513060,192495,2018-12-17,5,This is a keeper. Delicious Soup that both my ...
428498,454804,20713,2018-12-17,0,"Made this as gifts. Did 6, quart jars plus had..."
428499,1290903,131607,2018-12-18,5,This is a great recipe for a nice thin crispy ...


In [ ]:
print(f"# of recipes with images : {len(file_names)}")

users = sort_filter_ui['user_id'].unique().tolist()
items = sort_filter_ui['recipe_id'].unique().tolist()
print(f"number of users: {len(users)}, number of items: {len(items)}")

train_ = sort_filter_ui[:int(0.6*len(sort_filter_ui))]
valid_ = sort_filter_ui[int(0.6*len(sort_filter_ui)):int(0.7*len(sort_filter_ui))]
test_ = sort_filter_ui[int(0.7*len(sort_filter_ui)):]

u_tr = set(train_['user_id'].tolist())
u_va = set(valid_['user_id'].tolist())
u_te = set(test_['user_id'].tolist())
u_total = u_tr & u_te

filter_u_tr = train_[train_['user_id'].isin(u_total)].reset_index(drop=True)
filter_u_te = test_[test_['user_id'].isin(u_total)].reset_index(drop=True)
filter_u_va = valid_[valid_['user_id'].isin(u_total)].reset_index(drop=True)
print(f"train interaction count : {len(filter_u_tr)}, valid interaction count : {len(filter_u_va)}, test interaction count : {len(filter_u_te)}")

u_train = set(filter_u_tr['user_id'].tolist())
u_test = set(filter_u_te['user_id'].tolist())
u_valid = set(filter_u_va['user_id'].tolist())
print(f"train user count : {len(u_train)}, valid user count : {len(u_valid)}, test user count : {len(u_test)}")

i_tr = set(filter_u_tr['recipe_id'].tolist())
i_te = set(filter_u_te['recipe_id'].tolist())
i_va = set(filter_u_va['recipe_id'].tolist())
print(f"train item count : {len(i_tr)}, valid item count : {len(i_va)}, test item count : {len(i_te)}")

i_total = i_tr|i_va|i_te
print(f"number of users to use : {len(u_total)}")
print(f"number of items to use : {len(i_total)}")

In [ ]:
import ast

train_interaction = filter_u_tr
test_interaction = filter_u_te
valid_interaction = filter_u_va
print(f"train interaction count : {len(train_interaction)}, valid interaction count : {len(valid_interaction)}, test interaction count : {len(test_interaction)}")
print(train_interaction.columns)
print(test_interaction.columns)
print(valid_interaction.columns)

recipe_processed = raw_recipe[raw_recipe['id'].isin(i_total)].reset_index(drop=True)
print(f"number of recipes to use : {len(recipe_processed)}")
print(recipe_processed.columns)

# calories (#), total fat (PDV), sugar (PDV) , sodium (PDV) , protein (PDV) , saturated fat (PDV), carbohydrates (PDV)
def process_row_detailed(row):
    nutrition_types = ['calories', 'fat', 'sugar', 'sodium', 'protein', 'saturated_fat', 'carbohydrates']
    nutrition = ast.literal_eval(row['nutrition'])
    cooking_steps = ast.literal_eval(row['steps'])

    parts = []
    for n_type, value in zip(nutrition_types, nutrition):
        if int(value) == value:
            value = int(value)
        if n_type == 'calories':
            parts.append(f"calories: {value} kcal")
        else:
            parts.append(f"{n_type}: {value}% of Daily Value")
    
    nutrition_summary = ", ".join(parts)

    parts2 = []
    for idx, step in enumerate(cooking_steps):
        parts2.append(f"{idx+1}. {step}")
    cooking_steps_summary = "\n".join(parts2)

    ingredients = ", ".join(ast.literal_eval(row['ingredients']))
    
    return pd.Series({
        'id': row['id'],
        'name': row['name'],
        'ingredients': ingredients,
        'nutrition': nutrition_summary,
        'steps' : cooking_steps_summary
    })

drop_columns_detailed = ['minutes', 'contributor_id', 'submitted', 'tags', 'n_steps',  'description', 'n_ingredients']
recipe_drop_detailed = recipe_processed.drop(columns=drop_columns_detailed)

# Apply transformation
detailed_summary_recipe = recipe_drop_detailed.apply(process_row_detailed, axis=1)
print(detailed_summary_recipe.columns)
print(detailed_summary_recipe.head(3))

In [ ]:
import re
from tqdm import tqdm
from PIL import Image
from transformers import AutoProcessor, LlavaForConditionalGeneration

cache_dir = "../../data/jeehoshin/huggingface"
model_id = "llava-hf/llava-1.5-7b-hf"
model = LlavaForConditionalGeneration.from_pretrained(
    model_id, 
    cache_dir=cache_dir,
    torch_dtype=torch.float16, 
    low_cpu_mem_usage=True, 
).to(0)

processor = AutoProcessor.from_pretrained(
    model_id,
    cache_dir=cache_dir
)

def extract_summary_with_tags(generated_text):
    matches = re.search(r"<summary>(.*)", generated_text, re.DOTALL | re.IGNORECASE)
    if matches:
        text = matches.group(1).strip()
    else:
        text = generated_text.strip()  # fallback
    matches2 = re.search(r"^(.*?)</summary>", text, re.DOTALL | re.IGNORECASE)
    if matches2:
        return matches2.group(1).strip()
    else:
        return text.strip()  # fallback

def generate_batch_summary(model, processor, data_batch, device):
    images, prompts = [], []

    for recipe_id, recipe_name, ingredients, nutritions, directions in data_batch:
    # Image extensions in the directory: {'.jpeg', '.png', '.jpg', '.JPG', '.PNG'}
        image_path1 = f"../../data/jeehoshin/foodcom_dataset/images/{recipe_id}.jpeg"
        image_path2 = f"../../data/jeehoshin/foodcom_dataset/images/{recipe_id}.png"
        image_path3 = f"../../data/jeehoshin/foodcom_dataset/images/{recipe_id}.jpg"
        image_path4 = f"../../data/jeehoshin/foodcom_dataset/images/{recipe_id}.JPG"
        image_path5 = f"../../data/jeehoshin/foodcom_dataset/images/{recipe_id}.PNG"
        
        image_path = None
        for path in [image_path1, image_path2, image_path3, image_path4, image_path5]:
            if os.path.exists(path):
                image_path = path
                break

        try:
            image = Image.open(image_path).convert("RGB")  # Load local image
        except Exception as e:
            print(f"Failed to load image: {image_path}")
            image = Image.new("RGB", (224, 224), (255, 255, 255))  # fallback blank image
        
        images.append(image)

        prompt = f"""<image>
        You are a professional culinary analyst and recipe summarization assistant. Based on the provided image and detailed recipe information below, generate a rich and comprehensive summary of the dish.

        Focus on the following aspects:
        1. **Visual Presentation**: Describe the dish's appearance, including color, texture, portioning, and plating style.
        2. **Core Ingredients**: Highlight the main ingredients and any notable combinations or preparation methods.
        3. **Cooking Process**: Summarize the key steps in the cooking process, emphasizing any unique techniques or methods.
        4. **Flavor and Texture**: Provide an impression of the dish's flavor profile, aroma, and expected texture.
        5. **Nutritional Highlights**: Mention any relevant nutritional traits (e.g., high fiber, protein-rich, low in saturated fat).
        6. **Target Audience**: Suggest the type of person or dietary preference this dish may appeal to (e.g., health-conscious eaters, comfort food lovers).

        Recipe Details:
        - **Name**: {recipe_name}
        - **Ingredients**: {ingredients}
        - **Directions**: {directions}
        - **Nutrition Information**: {nutritions}

        <summary>"""

        prompts.append(prompt)
  
    inputs = processor(text = prompts, images = images, return_tensors="pt", padding=True).to(device)
    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=200)
    decoded = processor.batch_decode(output, skip_special_tokens=True)
    
    summaries = [extract_summary_with_tags(txt) for txt in decoded]

    return summaries

# Example usage
batch_size = 8
results = []
for i in tqdm(range(0, len(detailed_summary_recipe), batch_size)):  
    batch = detailed_summary_recipe.iloc[i:i+batch_size][['id', 'name', 'ingredients', 'nutrition', 'steps']].values.tolist()
    batch_results = generate_batch_summary(model, processor, batch, device=torch.device("cuda"))
    results.extend(batch_results)

print("Summaries generated.")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
100%|███████████████████████████████| 3739/3739 [17:24:16<00:00, 16.76s/it]

Summaries generated.


In [ ]:
print(len(results))

detailed_summary_recipe['summary'] = results
detailed_summary_recipe.drop(columns=['ingredients', 'nutrition', 'steps'], inplace=True)
detailed_summary_recipe.to_csv(data_path + "detailed_recipe_summaries.csv", index=False)

print("Summaries saved.")

29905
Summaries saved.


In [ ]:
detailed = pd.read_csv(data_path + "detailed_recipe_summaries.csv")
print(detailed)

for i in range(5):
    text = detailed.iloc[i]['summary']
    print(text)
    print()

           id                                               name  \
0       63986                   chicken lickin  good  pork chops   
1       43026                                     chile rellenos   
2       54100                           grilled  venison burgers   
3       25775  how i got my family to eat spinach  spinach ca...   
4       90921         i stole the idea from mirj  sesame noodles   
...       ...                                                ...   
29900  283432                 zuccuash bake  from nimz territory   
29901   96811                                            zucotte   
29902  455000                                       zulu cabbage   
29903  208429            zuppa toscana soup   olive garden clone   
29904  308080                          zydeco ya ya deviled eggs   

                                                 summary  
0      The dish is a delicious and hearty pork chop m...  
1      The dish is a delicious chile rellenos, featur...  
2     